In [1]:
import sympy as sp
from IPython.display import display, HTML
display(HTML("<style>.container { width:200% !important; }</style>"))

t = sp.Symbol('t')
q1 = sp.Function("q1")(t)
q2 = sp.Function("q2")(t)
q1_dot = sp.diff(q1, t)
q2_dot = sp.diff(q2, t)
q1_ddot = sp.diff(q1_dot, t)
q2_ddot = sp.diff(q2_dot, t)

# Generalized coordinates and velocities
q = [q1, q2]
q_dot = sp.Matrix([q1_dot, q2_dot])
q_ddot = sp.Matrix([q1_ddot, q2_ddot])

# Define the link lengths and masses
m1, m2, g = sp.symbols('m1 m2 g')
l1, l2, lc1, lc2 = sp.symbols('l1 l2 lc1 lc2')
J1, J2 = sp.symbols('J1 J2')

In [2]:
from sympy.printing.str import StrPrinter

class CustomStrPrinter(StrPrinter):
    def _print_Pow(self, expr):
        base, exp = expr.as_base_exp()
        if exp == 1:
            return self._print(base)
        elif exp == -1:
            return f"1/({self._print(base)})"
        else:
            return f"{self._print(base)}^{self._print(exp)}"

# Use the custom printer
def custom_pretty(expr):
    return CustomStrPrinter().doprint(expr)

In [3]:
# Actuator parameters
xa, ya = sp.symbols('xa ya') #x, y coordinate of actuator

# Rope connection length along joint 2
lr2 = sp.symbols('lr2')
kx = l1 * sp.cos(q1) + lr2 * sp.cos(q2)
ky = l1 * sp.sin(q1) + lr2 * sp.sin(q2)

k = sp.Matrix([[kx],
               [ky]])

dkdq1 = sp.diff(k, q1)
dkdq2 = sp.diff(k, q2)

R = sp.Matrix([[xa - kx],
               [ya - ky]])
Rx = R[0]
Ry = R[1]
r = R/R.norm()

A1 = r.dot(dkdq1)
A2 = r.dot(dkdq2)
A = sp.Matrix([[A1],
               [A2]])

A = sp.simplify(A)
print(custom_pretty(A))
print("\n")
sp.pprint(A, wrap_line=False)

Matrix([
[ l1*(lr2*sin(q1(t) - q2(t)) - xa*sin(q1(t)) + ya*cos(q1(t)))/Abs(l1*sin(q1(t)) + lr2*sin(q2(t)) - ya)^2 + Abs(l1*cos(q1(t)) + lr2*cos(q2(t)) - xa)^2^1/2],
[lr2*(-l1*sin(q1(t) - q2(t)) - xa*sin(q2(t)) + ya*cos(q2(t)))/Abs(l1*sin(q1(t)) + lr2*sin(q2(t)) - ya)^2 + Abs(l1*cos(q1(t)) + lr2*cos(q2(t)) - xa)^2^1/2]])


⎡            l₁⋅(lr₂⋅sin(q₁(t) - q₂(t)) - xa⋅sin(q₁(t)) + ya⋅cos(q₁(t)))             ⎤
⎢────────────────────────────────────────────────────────────────────────────────────⎥
⎢   _________________________________________________________________________________⎥
⎢  ╱                                      2                                        2 ⎥
⎢╲╱  │l₁⋅sin(q₁(t)) + lr₂⋅sin(q₂(t)) - ya│  + │l₁⋅cos(q₁(t)) + lr₂⋅cos(q₂(t)) - xa│  ⎥
⎢                                                                                    ⎥
⎢            lr₂⋅(-l₁⋅sin(q₁(t) - q₂(t)) - xa⋅sin(q₂(t)) + ya⋅cos(q₂(t)))            ⎥
⎢──────────────────────────────────────────────────────────────────

In [4]:
h = sp.atan2(Rx, Ry)
theta = sp.Matrix([[h],
                   [q2]])

Jh = theta.jacobian(q)
Jh = sp.simplify(Jh)
sp.pprint(Jh, wrap_line = False)

⎡       l₁⋅(-l₁ - lr₂⋅cos(q₁(t) - q₂(t)) + xa⋅cos(q₁(t)) + ya⋅sin(q₁(t)))               lr₂⋅(-l₁⋅cos(q₁(t) - q₂(t)) - lr₂ + xa⋅cos(q₂(t)) + ya⋅sin(q₂(t)))       ⎤
⎢───────────────────────────────────────────────────────────────────────────────  ───────────────────────────────────────────────────────────────────────────────⎥
⎢                                     2                                        2                                       2                                        2⎥
⎢(l₁⋅sin(q₁(t)) + lr₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + lr₂⋅cos(q₂(t)) - xa)   (l₁⋅sin(q₁(t)) + lr₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + lr₂⋅cos(q₂(t)) - xa) ⎥
⎢                                                                                                                                                                ⎥
⎣                                       0                                                                                1                                       ⎦


In [5]:
Jh_inv = Jh.inv()
Jh_invtrans = Jh_inv.transpose()

In [6]:
A_theta = Jh_invtrans * A
A_theta = sp.simplify(A_theta)
sp.pprint(A_theta, wrap_line = False)

⎡                                                                                 ⎛  2                                                                              2                                                 2     2⎞                        ⎤
⎢                       -(lr₂⋅sin(q₁(t) - q₂(t)) - xa⋅sin(q₁(t)) + ya⋅cos(q₁(t)))⋅⎝l₁  + 2⋅l₁⋅lr₂⋅cos(q₁(t) - q₂(t)) - 2⋅l₁⋅xa⋅cos(q₁(t)) - 2⋅l₁⋅ya⋅sin(q₁(t)) + lr₂  - 2⋅lr₂⋅xa⋅cos(q₂(t)) - 2⋅lr₂⋅ya⋅sin(q₂(t)) + xa  + ya ⎠                        ⎥
⎢                       ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────                       ⎥
⎢                                                    _________________________________________________________________________________                                                                                                                ⎥
⎢       

In [7]:
expr = lr2 * ((-l1*sp.sin(q1 - q2) - xa*sp.sin(q2) + ya*sp.cos(q2)) *
              (l1 + lr2*sp.cos(q1 - q2) - xa*sp.cos(q1) - ya*sp.sin(q1)) +
              (-lr2*sp.sin(q1 - q2) + xa*sp.sin(q1) - ya*sp.cos(q1)) *
              (l1*sp.cos(q1 - q2) + lr2 - xa*sp.cos(q2) - ya*sp.sin(q2)))

sp.pprint(expr, wrap_line=False)

lr₂⋅((-l₁⋅sin(q₁(t) - q₂(t)) - xa⋅sin(q₂(t)) + ya⋅cos(q₂(t)))⋅(l₁ + lr₂⋅cos(q₁(t) - q₂(t)) - xa⋅cos(q₁(t)) - ya⋅sin(q₁(t))) + (-lr₂⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₁(t)) - ya⋅cos(q₁(t)))⋅(l₁⋅cos(q₁(t) - q₂(t)) + lr₂ - xa⋅cos(q₂(t)) - ya⋅sin(q₂(t))))


In [8]:
l1 = 1
lr2 = 2
xa = 3
ya = 4
q1 = 0.1
q2 = 0.2
expr = lr2 * ((-l1*sp.sin(q1 - q2) - xa*sp.sin(q2) + ya*sp.cos(q2)) *
              (l1 + lr2*sp.cos(q1 - q2) - xa*sp.cos(q1) - ya*sp.sin(q1)) +
              (-lr2*sp.sin(q1 - q2) + xa*sp.sin(q1) - ya*sp.cos(q1)) *
              (l1*sp.cos(q1 - q2) + lr2 - xa*sp.cos(q2) - ya*sp.sin(q2)))
expr = sp.simplify(expr)
sp.pprint(expr, wrap_line=False)

2.45027466133665
